# Versiones de librerías
```
flask   = 1.1.2
numpy   = 1.20.1
pandas  = 1.2.4
pickle  = 4.0
sklearn = 0.24.1
sqlite3 = 2.6.0
```

# Configurar runtime

In [1]:
# Cargar librerías
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
import pickle

# Datos
Todas las tablas son vienen de [ENIGH 2020](https://www.inegi.org.mx/programas/enigh/nc/2020/#Microdatos).

## Lectura

1. Población

In [2]:
# Población
cols = ['folioviv','foliohog','numren','sexo','edad','hablaind','nivelaprob','hor_1','hor_2',
        'hor_4','hor_6','pop_insabi','segvol_2','segvol_3','hijos_sob']
p = pd.read_csv('../dat/poblacion.csv', usecols=cols, na_values=' ')

2. Hogares

In [3]:
# Hogares
cols = ['folioviv','foliohog','telefono','celular','tv_paga','conex_inte','num_auto','num_van',
        'num_pickup','num_moto','num_compu']
h = pd.read_csv('../dat/hogares.csv', usecols=cols)

3. Vivienda

In [4]:
# Viviendas
cols = ['folioviv','num_cuarto','tenencia','aire_acond','tot_resid']
v = pd.read_csv('../dat/viviendas.csv', usecols=cols)

4. Ingresos

In [5]:
# Ingresos
cols = ['folioviv','foliohog','numren','ing_tri']
i = pd.read_csv('../dat/ingresos.csv', usecols=cols)

## Consolidar
Consolidar datos en `df`

In [6]:
# Merge ingreso y población
df = pd.merge(i, p, how='left', on=['folioviv','foliohog','numren'])

# Agregar hogares
df = df.merge(h, how='left', on=['folioviv','foliohog'])

# Agregar viviendas
df = df.merge(v, how='left', on=['folioviv'])

## Limpieza

### Filtros

In [7]:
# Filtros poblacionales
df = df[df['foliohog'].eq(1) & df['edad'].ge(18) & df['nivelaprob'].notna()].copy()

### Modificación de columnas

In [8]:
# Ingreso mensual
df['ingreso'] = df['ing_tri'].div(3)

# Llenar NaNs
cols = ['hor_1','hor_2','hor_4','hor_6','segvol_3','hijos_sob']
df[cols] = df[cols].fillna(0)

# Traducir binarias a {0,1}
cols = ['sexo','pop_insabi','telefono','celular','tv_paga','conex_inte','aire_acond']
df[cols] = df[cols] - 1
df['hablaind'] = df['hablaind'].replace(2,0)
df[['segvol_2','segvol_3']] = (df[['segvol_2','segvol_3']] > 0)

# Categóricas a enteros
cat = ['sexo','hablaind','nivelaprob','pop_insabi','segvol_2','segvol_3','telefono','celular',
       'tv_paga','conex_inte','tenencia','aire_acond']
df[cat] = df[cat].apply(lambda x: x.astype(int))

# Número de vehículos
df['num_auto'] = df[['num_auto','num_van','num_pickup']].sum(axis=1)

# Quitar columnas inútiles
df.drop(columns=['ing_tri','num_van','num_pickup'], inplace=True)

#### Multicategóricas
1. Mapping de `nivelaprob`

        df.groupby('nivelaprob')['ingreso'].mean().sort_values()

| Máximo grado | Valor INEGI |  Input   | Map |
|--------------|-------------|----------|-----|
|Ninguno       |            0|secundaria|  0  |
|Preescolar    |            1|secundaria|  0  | 
|Primaria      |            2|secundaria|  0  |
|Secundaria    |            3|secundaria|  0  |
|Preparatoria  |            4|   prepa  |  1  |
|Normal        |            5|  carrera |  2  |
|Técnica       |            6|  carrera |  2  |
|Profesional   |            7|  carrera |  2  |
|Maestría      |            8| maestria |  3  |
|Doctorado     |            9| doctorado|  4  |

1. Mapping de `tenencia`

        df.groupby('tenencia')['ingreso'].mean().sort_values()

| Tenencia | valor INEGI |   Map    | Map |
|----------|-------------|----------|-----|
|Rentada   |            1|Rentada   |  2  |
|Prestada  |            2|Otra      |  0  |
|Propia    |            3|Propia    |  3  |
|Pagando   |            4|Hipotecada|  1  |
|Litigio   |            5|Otra      |  0  |
|Otra      |            6|Otra      |  0  |

In [9]:
# Ordenar nivelaprob
d_educ = {0:0,1:0,2:0,3:0, # Secundaria o menos
          4:1,             # Prepa
          5:2,6:2,7:2,     # Carrera
          8:3,             # Maestría
          9:4}             # Doctorado
df['nivelaprob'] = df['nivelaprob'].replace(d_educ)

# Ordenar tenencia
d_casa = {2:0,5:0,6:0,     # Otra
          4:1,             # Hipotecada
          1:2,             # Rentada
          3:3}             # Propia

# Entrenar modelo
Pasos a seguir:
1. Partir datos en `train` y `test`
1. Entrenar un [Gradient Boosting Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) sobre datos de entrenamiento y usando todos los atributos
1. Verificar que el desempeño sea similar entre `train` y `test`
1. Quedarnos solo con los mejores 10 atributos
1. Volver a entrenar con el 100% de la muestra

## 1. Dividir en train y test

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, 'sexo':'tot_resid'], df['ingreso'],
                                                    test_size=0.1, random_state=123)

## 2. Entrenar sobre datos de entrenamiento

In [11]:
# Hiperparámetros
gb = GradientBoostingRegressor(loss='ls', learning_rate=0.03, n_estimators=300,
                               max_depth=3, random_state=123)
# Fit
gb.fit(X_train.values, y_train.values)

GradientBoostingRegressor(learning_rate=0.03, n_estimators=300,
                          random_state=123)

## 3. Desempeño entre datos de entrenamiento y prueba

In [12]:
# Performance en train
print('Train:', round(gb.score(X_train.values, y_train.values), 3))
# Performance en test
print('Test: ', round(gb.score(X_test.values, y_test.values), 3))

Train: 0.155
Test:  0.141


No hay _tanta_ diferencia entre los scores.
- 0.155
- 0.141

## 4. Conservar top 10 features

In [13]:
# Visualizar top 10 atributos
t_gb = pd.DataFrame({'feature':X_train.columns, 'importance':gb.feature_importances_ * 100})
t_gb = t_gb.sort_values('importance', ascending=False).reset_index(drop=True)
t_gb.round(1).head(10)

# Conservar top 10 atributos
x = t_gb['feature'].values.tolist()[:10]

## 5. Reentrenar sobre toda la muestra usando 10 atributos

In [14]:
# Fit sobre 100%
gb.fit(df[sorted(x)].values, df['ingreso'].values)

GradientBoostingRegressor(learning_rate=0.03, n_estimators=300,
                          random_state=123)

In [15]:
# Nuevo score
round(gb.score(df[sorted(x)].values, df['ingreso'].values), 2)

0.14

El score no subió mucho desde que entrenamos con solo 90%.

- 0.14

Por ende conservaremos este modelo.

# Exportar modelo

In [18]:
pickle.dump(gb, open('../model/gbr.pkl', 'wb'))

# Exportar datos

In [17]:
# Agregar `pid` (person id)
df = df[['ingreso'] + sorted(x)]
df.reset_index(inplace=True)

# Renombrar columnas
df.columns = ['pid','ingreso','internet','edad','hrs_trab','educacion','num_autos','num_compu',
              'num_cuartos','seguro_med','mujer','resid']

# Exportar
df.to_csv('../dat/df.csv', index=False)